# Project 2: Ask Your Data

Build your own "Julius AI" — ask questions in plain English, and an AI agent writes and runs Python code to answer them. The dataset is the top ~953 Spotify songs of 2023, with columns covering track info, platform metrics, and audio features.

The main lesson: **the system prompt is 80% of the work**. You'll see a naive prompt fail, then iterate until the agent reliably answers questions about the data.

| What you'll learn | Where it comes from |
|-------------------|--------------------|  
| **Code Tool** | New — the agent generates and executes Python code |
| **System prompt engineering** | Builds on prompt engineering (Appendix D, E) |
| **Agent with tools** | Builds on agents (Ch 7) and tool calling (Ch 8) |

**Workflow in this chapter:**

| File | What it does | GitHub Link |
|------|-------------|-------------|
| `11_ask_your_data.json` | Chat agent that writes + runs Python code against the Spotify CSV | [View](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/book/_static/workflows/11_ask_your_data.json) |

**Credentials needed:** OpenRouter API key (Settings → Credentials).

---

## The Dataset

We use the [Most Streamed Spotify Songs 2023](https://www.kaggle.com/datasets/nelgiriyewithana/top-spotify-songs-2023) dataset — ~953 songs, 24 columns.

**Source URL (used by the agent):**
```
https://raw.githubusercontent.com/zero-to-mastery/file-io/main/spotify-2023.csv
```

The columns fall into three groups:

| Group | Columns | Description |
|-------|---------|-------------|
| **Track info** | `track_name`, `artist(s)_name`, `artist_count`, `released_year`, `released_month`, `released_day` | Song identity and release date |
| **Platform metrics** | `in_spotify_playlists`, `in_spotify_charts`, `streams`, `in_apple_playlists`, `in_apple_charts`, `in_deezer_playlists`, `in_deezer_charts`, `in_shazam_charts` | Popularity across platforms |
| **Audio features** | `bpm`, `key`, `mode`, `danceability_%`, `valence_%`, `energy_%`, `acousticness_%`, `instrumentalness_%`, `liveness_%`, `speechiness_%` | Musical characteristics (0–100 scale) |

### Preview (first 5 rows)

| track_name | artist(s)_name | artist_count | released_year | streams | bpm | key | mode | danceability_% |
|---|---|---|---|---|---|---|---|---|
| Seven (feat. Latto) (Explicit Ver.) | Latto, Jung Kook | 2 | 2023 | 141381703 | 125 | B | Major | 80 |
| LALA | Myke Towers | 1 | 2023 | 133716286 | 92 | C# | Major | 71 |
| vampire | Olivia Rodrigo | 1 | 2023 | 140003974 | 138 | F | Major | 51 |
| Cruel Summer | Taylor Swift | 1 | 2019 | 800840817 | 170 | A | Major | 55 |
| WHERE SHE GOES | Bad Bunny | 1 | 2023 | 303236322 | 144 | A | Minor | 65 |

**Gotchas to note:**
- `streams` is stored as a **string**, not a number — needs conversion
- Column names have special characters: `artist(s)_name`, `danceability_%`
- `in_deezer_playlists` and `in_shazam_charts` may contain commas or be empty
- There is **no genre column** and **no song duration column**

---

## The Workflow

One workflow, three nodes:

```
┌──────────────────────┐     ┌──────────────────┐     ┌──────────┐
│ When chat message    │────▶│ Data Analyst     │────▶│ Output   │
│ received             │     │ Agent            │     │          │
└──────────────────────┘     └──────────────────┘     └──────────┘
                                      ┊ sub-nodes
                             ┌────────┴────────┐
                             ┊                 ┊
                        Chat Model      Python Code Tool
```

- **Chat Trigger** — students type questions naturally
- **AI Agent** — receives the question, generates Python code, interprets results, answers in plain English
- **Python Code Tool** — executes the generated pandas code against the CSV

**File:** [`11_ask_your_data.json`](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/book/_static/workflows/11_ask_your_data.json)

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/11_ask_your_data.json
> ```
>
> **Download:** {download}`11_ask_your_data.json <_static/workflows/11_ask_your_data.json>`

::::{dropdown} 🛠️ Build this workflow from scratch (step-by-step)
:color: secondary

### Step 1: Create a new workflow

1. Click **Workflows** → **Add Workflow**
2. Rename it to "Ask Your Data (Spotify)"

### Step 2: Add the Chat Trigger

1. Delete the default Manual Trigger
2. Add **Chat Trigger** (search for "When chat message received")
3. In Settings, set **Response Mode** to `Last Node`

### Step 3: Add the AI Agent

1. Add **AI Agent** → rename to `Data Analyst Agent`
2. Configure:
   - **Source for Prompt**: `Define below`
   - **Prompt** (Expression): `{{ $json.chatInput }}`
   - Open **Options** → add **System Message** (paste the full system prompt from the "Final System Prompt" section below)

### Step 4: Add the Chat Model (sub-node)

1. Click **+ Chat Model** at the bottom of the AI Agent node
2. Select **OpenRouter Chat Model** (or OpenAI, etc.)
3. Choose your credential
4. Model: `openai/gpt-4o-mini`

### Step 5: Add the Code Tool (sub-node)

1. Click **+ Tool** at the bottom of the AI Agent node
2. Select **Code Tool**
3. Configure:

| Setting | Value |
|---------|-------|
| **Name** | `python_code` |
| **Description** | `Execute Python code to analyze the Spotify dataset. The code should use pandas to query, filter, aggregate, or compute statistics on the data. Always print the final result.` |
| **Language** | `Python` |

4. Leave the default code as-is — the agent replaces it at runtime

### Step 6: Add Output

1. Add **Edit Fields** → rename to `Output`
2. Add field:
   - Name: `output`
   - Value (Expression): `{{ $json.output }}`

### Step 7: Connect the main nodes

```
When chat message received → Data Analyst Agent → Output
```

### Step 8: Test

1. Click **Chat** in the workflow UI
2. Ask: "Which song has the most streams?"
3. Watch the agent write and execute Python code, then return the answer

::::

### Node-by-Node Walkthrough

| Node | Type | What it does |
|------|------|-------------|
| **When chat message received** | Chat Trigger | Receives the user's question via the chat UI |
| **Data Analyst Agent** | AI Agent | Generates Python code, calls the Code Tool, interprets results |
| **Output** | Set | Saves `{{ $json.output }}` as `output` for the chat response |

**Sub-nodes (dotted lines to the Agent):**

| Sub-node | Type | Purpose |
|----------|------|--------|
| **OpenRouter Chat Model** | Chat Model | Provides the LLM |
| **Python Code Tool** | Code Tool | Executes the Python code the agent writes |

### Data Flow

```
INPUT                          AGENT ACTIONS                        OUTPUT
─────                          ─────────────                        ──────
Chat: { chatInput: "Which song has the most streams?" }
    ↓
Agent:
  1. Writes Python code:
     df['streams'] = pd.to_numeric(df['streams'], errors='coerce')
     top = df.nlargest(1, 'streams')[['track_name','artist(s)_name','streams']]
     print(top.to_string())
  2. Calls Python Code Tool → gets printed output
  3. Interprets result and writes a plain English answer
    ↓
Output: { output: "The song with the most streams is 'Blinding Lights' by The Weeknd with ~3.7 billion streams." }
```

---

## The System Prompt

This is the core of the project. The system prompt is what makes the agent work — or fail. We'll iterate through three versions to show why each piece matters.

### V1: The Naive Prompt

What students might try first:

```
You are a data analyst. Answer questions about Spotify data.
```

**What goes wrong:**
- The agent doesn't know the column names → generates `df['artist']` instead of `df['artist(s)_name']`
- It assumes `streams` is a number → crashes on string comparison
- It doesn't know the CSV URL → tries to read a local file that doesn't exist
- It invents columns like `genre` or `duration` that don't exist

The agent has the right tools but no context. It's like hiring an analyst and handing them an unlabeled spreadsheet.

### V2: Add the Schema

Add column names, types, and descriptions:

```
You are a data analyst. Answer questions about a Spotify dataset by writing Python code.

Always load the CSV from:
https://raw.githubusercontent.com/zero-to-mastery/file-io/main/spotify-2023.csv

Columns:
- track_name (str): Song title
- artist(s)_name (str): Artist name(s)
- artist_count (int): Number of artists
- released_year (int): Release year
- released_month (int): Release month
- released_day (int): Release day
- in_spotify_playlists (int): Spotify playlist count
- in_spotify_charts (int): Spotify chart presence
- streams (str): Total streams — WARNING: stored as string
- bpm (int): Beats per minute
- key (str): Musical key
- mode (str): Major or Minor
- danceability_% (int): 0-100
- valence_% (int): 0-100
- energy_% (int): 0-100
- acousticness_% (int): 0-100
- instrumentalness_% (int): 0-100
- liveness_% (int): 0-100
- speechiness_% (int): 0-100
```

**Improvement:** The agent now uses correct column names and knows to convert `streams`. But it still makes mistakes — it doesn't know about encoding issues (`latin-1`), doesn't handle commas in `in_deezer_playlists`, and sometimes returns raw DataFrames instead of readable answers.

### V3: Add Sample Data + Rules

The final version adds three things:

1. **Sample data** — a `.head(5)` preview so the agent sees real values
2. **Explicit rules** — how to handle edge cases
3. **Output format** — always `print()` a readable answer

This is the version that works well. Let's look at it in full.

---

### Final System Prompt

This is the complete system prompt used in the workflow, with annotations explaining each section.

**Data Analyst Agent — System Message:**
```
You are a data analyst. You answer questions about a Spotify dataset by writing Python code.

## Dataset
The CSV is loaded from this URL:
```

```
https://raw.githubusercontent.com/zero-to-mastery/file-io/main/spotify-2023.csv
```

```
Always start your code with:
```

```python
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/zero-to-mastery/file-io/main/spotify-2023.csv', encoding='latin-1')
```

↑ **Why this matters:** The CSV uses Latin-1 encoding (some song names have accented characters). Without `encoding='latin-1'`, the code crashes on rows with special characters.

```
## Schema (24 columns)
| Column | Type | Description |
|--------|------|-------------|
| track_name | str | Song title |
| artist(s)_name | str | Artist name(s), comma-separated |
| artist_count | int | Number of contributing artists |
| released_year | int | Release year |
| released_month | int | Release month (1-12) |
| released_day | int | Release day (1-31) |
| in_spotify_playlists | int | Number of Spotify playlists the song appears in |
| in_spotify_charts | int | Presence in Spotify charts |
| streams | str | Total streams (WARNING: this is a string, not a number) |
| in_apple_playlists | int | Number of Apple Music playlists |
| in_apple_charts | int | Presence in Apple Music charts |
| in_deezer_playlists | str | Number of Deezer playlists (may contain commas) |
| in_deezer_charts | int | Presence in Deezer charts |
| in_shazam_charts | str | Presence in Shazam charts (may be empty) |
| bpm | int | Beats per minute |
| key | str | Musical key (C, C#, D, ..., B) — may be empty |
| mode | str | Major or Minor |
| danceability_% | int | How danceable (0-100) |
| valence_% | int | How positive/happy (0-100) |
| energy_% | int | How energetic (0-100) |
| acousticness_% | int | How acoustic (0-100) |
| instrumentalness_% | int | How instrumental (0-100) |
| liveness_% | int | Presence of live audience (0-100) |
| speechiness_% | int | Amount of spoken words (0-100) |
```

↑ **Why this matters:** The schema gives the agent exact column names (with special characters like parentheses and `%`) and types. Without it, the agent guesses wrong names.

```
## Sample data (first 5 rows)
| track_name | artist(s)_name | artist_count | released_year | streams | bpm | key | mode | danceability_% |
|---|---|---|---|---|---|---|---|---|
| Seven (feat. Latto) (Explicit Ver.) | Latto, Jung Kook | 2 | 2023 | 141381703 | 125 | B | Major | 80 |
| LALA | Myke Towers | 1 | 2023 | 133716286 | 92 | C# | Major | 71 |
| vampire | Olivia Rodrigo | 1 | 2023 | 140003974 | 138 | F | Major | 51 |
| Cruel Summer | Taylor Swift | 1 | 2019 | 800840817 | 170 | A | Major | 55 |
| WHERE SHE GOES | Bad Bunny | 1 | 2023 | 303236322 | 144 | A | Minor | 65 |
```

↑ **Why this matters:** Sample data shows the agent what real values look like. It sees that `streams` looks like a number but is actually a string. It sees that `artist(s)_name` can contain commas. It sees the scale of values.

```
## Rules
1. The `streams` column is stored as a string. Always convert it: pd.to_numeric(df['streams'], errors='coerce')
2. Column names have special characters — use exact names: artist(s)_name, danceability_%, etc.
3. When the question CANNOT be answered with the available columns, say so clearly.
   For example, there is no genre column and no song duration column.
4. Always print your final answer as a readable string — not raw DataFrames. Use print() to return the result.
5. Keep code simple and correct. Use pandas for all analysis.
6. If the user asks for a chart or plot, use matplotlib and save to a file,
   but note that the output is text-only.
```

↑ **Why this matters:** Rules handle the known edge cases. Rule 1 prevents the most common crash. Rule 3 prevents hallucination. Rule 4 ensures the agent returns human-readable answers.

### Key Takeaway

The system prompt went from 1 line to ~50 lines. Each addition fixed a real failure mode:

| What we added | What it fixed |
|---------------|---------------|
| CSV URL + load code | Agent couldn't find the data |
| Column schema | Agent used wrong column names |
| Type annotations | Agent treated strings as numbers |
| Sample data | Agent didn't understand data format |
| `streams` warning | Most common crash (string → numeric) |
| "Say so clearly" rule | Agent invented answers for missing columns |
| `print()` rule | Agent returned raw DataFrames |

**The system prompt IS the engineering.** The workflow itself is just three nodes.

---

## Example Questions

Try these in the chat to see the agent in action. They're ordered by difficulty.

### Simple Lookups

| Question | What the agent does |
|----------|--------------------|
| "Which song has the most streams?" | Converts `streams` to numeric, finds the max, returns track name + artist |
| "How many songs did Bad Bunny release?" | Filters `artist(s)_name` containing "Bad Bunny", counts rows |
| "List all songs released in January 2023" | Filters by `released_year == 2023` and `released_month == 1` |

### Aggregations

| Question | What the agent does |
|----------|--------------------|
| "What's the average BPM of songs in Major vs Minor mode?" | Groups by `mode`, computes `bpm.mean()` |
| "Which artist appears most often in the dataset?" | Splits `artist(s)_name` by comma, counts occurrences |
| "What's the average danceability of the top 50 most-streamed songs?" | Sorts by streams, takes top 50, computes mean of `danceability_%` |

### Interesting Analysis

| Question | What the agent does |
|----------|--------------------|
| "Is there a correlation between danceability and streams?" | Computes correlation coefficient between `danceability_%` and `streams` |
| "What month has the most song releases?" | Groups by `released_month`, counts, finds the max |
| "Show me the top 5 most energetic songs with low acousticness" | Filters `acousticness_% < 20`, sorts by `energy_%`, takes top 5 |

### Edge Cases (things that fail or need creativity)

| Question | What should happen |
|----------|-------------------|
| "What's the most popular genre?" | No `genre` column — the agent should say so |
| "How long is the average song?" | No `duration` column — the agent should recognize this |
| "Which song was #1 on Spotify?" | The `in_spotify_charts` column doesn't directly mean rank #1 — needs interpretation |
| "Compare Taylor Swift and Drake" | Requires filtering by artist name (substring match), then comparing metrics |

---

## What to Observe + Key Takeaways

After running several questions, here's what to notice:

**1. The system prompt is 80% of the work**

The workflow is just three nodes. All the intelligence comes from the system prompt — the schema, the sample data, the rules. If you change the dataset, you only need to update the prompt.

**2. Schema + sample data > long instructions**

Showing the agent 5 rows of real data teaches it more than paragraphs of instructions. The agent sees data types, value ranges, and formatting issues directly.

**3. The agent retries on errors**

If the generated code crashes (e.g., wrong column name), the agent sees the error message and writes corrected code. This is the reflection pattern from Chapter 6 — happening naturally inside the agent loop.

**4. Some questions can't be answered**

A good agent says "I can't answer that with this data" instead of making something up. The rule "say so clearly" in the system prompt makes this reliable.

**5. This pattern works with ANY CSV**

The workflow is generic. Swap the CSV URL, update the schema and sample data in the system prompt, and you have a data analyst for any dataset.

---

## Try It Yourself

### Challenge 1: Use Your Own Data

1. Find a CSV dataset you care about (Kaggle is a good source)
2. Host it at a public URL (or use a raw GitHub link)
3. Update the system prompt:
   - Replace the CSV URL
   - Replace the schema table with your columns
   - Replace the sample data with your `.head(5)`
   - Update the rules for your data's quirks
4. Test with progressively harder questions

### Challenge 2: Multi-Dataset Agent

Add a second CSV (e.g., a 2024 Spotify dataset or a different music platform). Update the system prompt to describe both datasets. Ask questions that span both:
- "Which artists appear in both the 2023 and 2024 datasets?"
- "How did average BPM change between 2023 and 2024?"

### Challenge 3: Add Guardrails

The Code Tool can run arbitrary Python. Add rules to the system prompt to prevent:
- Code that writes to disk (no `to_csv`, `open()`, etc.)
- Code that imports dangerous modules (`os`, `subprocess`, `sys`)
- Code that makes network requests beyond loading the CSV

This connects back to Chapter 9 (Guardrails & Safety).

### Challenge 4: Better Error Messages

When the agent's code fails, it sees the raw Python traceback. Add a rule to the system prompt: "If your code fails, explain what went wrong in plain English before retrying." Compare how the agent behaves with and without this rule.